# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file="../WeatherPy/weather_data.csv"
data_file=pd.read_csv(file)
# data_file_df.dropna(inplace = True)
data_file_df=pd.DataFrame(data_file)
data_file_df.set_index(data_file_df.columns[0])
data_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.6000,168.3333,7.39,81,100,4.20,NZ,1662901707
1,Bengkulu,-3.8004,102.2655,25.64,78,96,1.06,ID,1662901911
2,Bredasdorp,-34.5322,20.0403,15.41,54,100,7.30,ZA,1662902001
3,Vaini,-21.2000,-175.2000,23.83,77,9,5.76,TO,1662902001
4,Tsimmerbude,54.6750,20.1347,16.03,51,100,7.61,RU,1662901709


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations=data_file_df[['Lat','Lng']]
weight=data_file_df['Humidity']

# Plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)
# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities to fit weather conditions.

# Drop any rows that don't satisfy all three conditions. You want to be sure that the weather is ideal.
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

filtered_city_data = data_file_df.loc[(data_file_df["Max Temp"] < 27) & (data_file_df["Max Temp"] > 21) & (data_file_df["Wind Speed"] < 4.5) & (data_file_df["Cloudiness"] == 0)]
filtered_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,Lompoc,34.6391,-120.4579,22.51,85,0,1.54,US,1662902003
25,Makakilo City,21.3469,-158.0858,24.18,83,0,2.06,US,1662902007
27,Van,38.4924,43.3831,24.64,22,0,2.57,TR,1662901726
88,Port Keats,-14.2500,129.5500,24.75,86,0,2.07,AU,1662901763
141,Celano,42.0855,13.5398,26.45,41,0,1.79,IT,1662901961
165,Hilo,19.7297,-155.0900,22.93,73,0,3.09,US,1662901817
168,Kahului,20.8947,-156.4700,26.02,68,0,2.68,US,1662901819
195,Plymouth,41.8334,-70.8328,23.26,75,0,1.54,US,1662902054
204,San Quintín,30.4833,-115.9500,22.43,87,0,2.21,MX,1662901837
218,Ambovombe,-25.1667,46.0833,25.42,52,0,4.48,MG,1662901844


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=filtered_city_data
hotel_df["Hotel Name"] = ""
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


C:\Users\User\AppData\Local\Temp\ipykernel_9832\1895811854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Retrieving Results for Index 13: Lompoc.
Closest hotel is Holiday Inn Express Lompoc, an IHG Hotel.
------------
Retrieving Results for Index 25: Makakilo City.
Closest hotel is Marriott's Ko Olina Beach Club.
------------
Retrieving Results for Index 27: Van.
Closest hotel is Elite World Van Hotel.
------------
Retrieving Results for Index 88: Port Keats.
Closest hotel is TDC Village (Hundred Man Camp).
------------
Retrieving Results for Index 141: Celano.
Closest hotel is Lory.
------------
Retrieving Results for Index 165: Hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 168: Kahului.
Closest hotel is Maui Seaside Hotel.
------------
Retrieving Results for Index 195: Plymouth.
Closest hotel is Rodeway Inn Middleboro-Plymouth.
------------
Retrieving Results for Index 204: San Quintín.
Closest hotel is Don Eddie's Sport Fishing Center.
------------
Retrieving Results for Index 218: Ambovombe.
Closest hotel is RELAHARA.
------------
Retrieving Res

In [9]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,Lompoc,34.6391,-120.4579,22.51,85,0,1.54,US,1662902003,"Holiday Inn Express Lompoc, an IHG Hotel"
25,Makakilo City,21.3469,-158.0858,24.18,83,0,2.06,US,1662902007,Marriott's Ko Olina Beach Club
27,Van,38.4924,43.3831,24.64,22,0,2.57,TR,1662901726,Elite World Van Hotel
88,Port Keats,-14.2500,129.5500,24.75,86,0,2.07,AU,1662901763,TDC Village (Hundred Man Camp)
141,Celano,42.0855,13.5398,26.45,41,0,1.79,IT,1662901961,Lory


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

SyntaxError: invalid syntax (724339415.py, line 9)